# WRFcast Tutorial

This tutorial will walk through forecast data from your own WRF forecast model data using the wrfcast.py module within pvlib.

This tutorial has been tested against the following package versions:
* Python 
* IPython 
* pandas 
* matplotlib 
* netcdf4 1.4.2 

It should work with other Python and Pandas versions. It requires pvlib >= 0.3.0 and IPython >= 3.0.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, November 2019

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf
import pvlib
from pvlib.wrfcast import WRF

In [3]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

In [48]:
netcdf_data = netCDF4.Dataset(datapath)

In [65]:
query_variables = [
            'T2', # T2
            'U10',
            'V10',
            'CLDFRA',
            'SWDDNI',
            'SWDDIF'
            ]
first = True
for key in query_variables:
    var = wrf.getvar(netcdf_data, key)
    if first:
        solar_data = var
        first = False
    else:
        solar_data = xr.merge([solar_data, var])


In [66]:
solar_data

<xarray.Dataset>
Dimensions:  (bottom_top: 35, south_north: 191, south_north_stag: 192, west_east: 191, west_east_stag: 192)
Coordinates:
    XLONG    (south_north, west_east) float32 -95.43335 -95.308044 ... -64.11832
    XLAT     (south_north, west_east) float32 30.952759 30.950832 ... 47.507103
    XTIME    float32 780.0
    Time     datetime64[ns] 2011-01-24T01:00:00
Dimensions without coordinates: bottom_top, south_north, south_north_stag, west_east, west_east_stag
Data variables:
    T2       (south_north, west_east) float32 287.62427 287.5925 ... 263.8601
    U        (bottom_top, south_north, west_east_stag) float32 0.55619085 ... 4.416398
    V        (bottom_top, south_north_stag, west_east) float32 1.0613657 ... 10.647256
    CLDFRA   (bottom_top, south_north, west_east) float32 0.0 0.0 ... 0.0 0.0
    SWDDNI   (south_north, west_east) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SWDDIF   (south_north, west_east) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0